In [18]:
import torch

In [19]:
from torch.autograd import Variable

In [20]:
import cv2

In [21]:
from data import BaseTransform, VOC_CLASSES as labelmap

In [22]:
from ssd import build_ssd

In [23]:
import imageio

In [24]:
def detect(frame, net, transform):
    height, width= frame.shape[:2]
    frame_t = transform(frame)[0]
    x = torch.from_numpy(frame_t).permute(2,0,1)
    x=Variable(x.unsqueeze(0))
    y = net(x)
    detections = y.data
    #print(type(detections))
    scale = torch.Tensor([width,height,width,height])
    #print(scale)
    
    for i in range(detections.size(1)):
        j=0
        while detections[0,i,j,0]>=0.6:
            pt = (detections[0,i,j,1:] * scale).numpy()
            cv2.rectangle(frame, (int(pt[0]),int(pt[1])),(int(pt[2]),int(pt[3])),(255,0,0),2)
            cv2.putText(frame, labelmap[i-1],(int(pt[0]),int(pt[1])),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),2,cv2.LINE_AA)
            j+=1
    return frame
                        
net = build_ssd('test')
net.load_state_dict(torch.load('ssd300_mAP_77.43_v2.pth',map_location=lambda storage,loc:storage))
transform =BaseTransform(net.size,(104/256.0,117/256.0,123/256.0))
reader=imageio.get_reader('funny_dog.mp4')
fps=reader.get_meta_data()['fps']
writer=imageio.get_writer('output.mp4',fps=fps)
for i,frame in enumerate(reader):
    frame=detect(frame, net.eval(),transform)
    writer.append_data(frame)
    print(i)
writer.close()

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1920, 1080) to (1920, 1088) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
